In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc

In [4]:
sc = SparkContext()

In [5]:
spark = SparkSession(sc)

In [6]:
# Instance StreamingContext with 10s
ssc = StreamingContext(sc, 10)

H:\Spark\spark-3.4.1-bin-hadoop3\python\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [7]:
socket_stream = ssc.socketTextStream("127.0.0.1", 9998)

In [8]:
lines = socket_stream.window(20)

In [9]:
from collections import namedtuple
fields = ("tag", "count")
Tweet = namedtuple("Tweet", field_names=fields)

In [10]:
(lines.flatMap(lambda text: text.split(" "))  # Divide o texto em palavras
  .filter(lambda word: word.lower().startswith("#"))  # Filtra apenas hashtags
  .map(lambda word: (word.lower(), 1))  # Converte para minúsculas e adiciona contagem
  .reduceByKey(lambda a, b: a + b)  # Soma as ocorrências
  .map(lambda rec: Tweet(rec[0], rec[1]))  # Converte para um objeto Tweet
  .foreachRDD(lambda rdd: rdd.toDF().sort(desc("count"))  # Converte para DataFrame e ordena
              .limit(10).createOrReplaceTempView("tweets"))  # Cria a tabela temporária
)

In [11]:
# TweetRead running
ssc.start()

In [12]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [13]:
count = 0
while count < 10:
    time.sleep(3)
    top_10_tweets = spark.sql('SELECT tag, count FROM tweets')  # Usando SparkSession
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize=(10, 8))
    sns.barplot(x="count", y="tag", data=top_10_df)
    plt.show()
    count += 1

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `tweets` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets], [], false
